In [ ]:
!git clone https://github.com/kkothari2001/twAIn

Cloning into 'twAIn'...
remote: Enumerating objects: 498, done.
remote: Counting objects: 100% (498/498), done.
remote: Compressing objects: 100% (405/405), done.
remote: Total 498 (delta 72), reused 460 (delta 46), pack-reused 0
Receiving objects: 100% (498/498), 38.74 MiB | 27.28 MiB/s, done.
Resolving deltas: 100% (72/72), done.


In [ ]:
!mv "twAIn/books/Adventure/long/robinson_crusoe.txt" "story.txt"

In [ ]:
from __future__ import print_function
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Input, Flatten, Bidirectional,Embedding
from keras.layers.normalization import BatchNormalization
from tensorflow.keras import regularizers
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.metrics import categorical_accuracy
import numpy as np
import random
import sys
import os
import time
import codecs
import collections
from six.moves import cPickle
import re

import spacy
nlp = spacy.load('en')

In [ ]:
def preprocess_text(text):
    # Remove punctuations and numbers
    corpus = re.sub('[^a-zA-Z.,!]', ' ', text)

    # Single character removal
    corpus = re.sub(r"\s+[a-zA-Z]\s+", ' ', corpus)

    # Removing multiple spaces
    corpus = re.sub(r'\s+', ' ', corpus)

    return corpus.lower()

def create_wordlist(doc):
    wl = []
    for word in doc:
        if word.text not in ("\n","\n\n",'\u2009','\xa0'):
            wl.append(word.text.lower())
    return wl


#read data
with codecs.open("story.txt", "r") as f:
  data = f.read()

#create sentences
preprocess_data = preprocess_text(str(data))

doc = nlp(preprocess_data)

word_list = create_wordlist(doc)

word_list

[' ',
 'the',
 'project',
 'gutenberg',
 'ebook',
 ',',
 'the',
 'life',
 'and',
 'adventures',
 'of',
 'robinson',
 'crusoe',
 ',',
 'by',
 'daniel',
 'defoe',
 'this',
 'ebook',
 'is',
 'for',
 'the',
 'use',
 'of',
 'anyone',
 'anywhere',
 'in',
 'the',
 'united',
 'states',
 'and',
 'most',
 'other',
 'parts',
 'of',
 'the',
 'world',
 'at',
 'no',
 'cost',
 'and',
 'with',
 'almost',
 'no',
 'restrictions',
 'whatsoever',
 '.',
 'you',
 'may',
 'copy',
 'it',
 ',',
 'give',
 'it',
 'away',
 'or',
 're',
 'use',
 'it',
 'under',
 'the',
 'terms',
 'of',
 'the',
 'project',
 'gutenberg',
 'license',
 'included',
 'with',
 'this',
 'ebook',
 'or',
 'online',
 'at',
 'www.gutenberg.org',
 '.',
 'if',
 'you',
 'are',
 'not',
 'located',
 'in',
 'the',
 'united',
 'states',
 ',',
 'you',
 'll',
 'have',
 'to',
 'check',
 'the',
 'laws',
 'of',
 'the',
 'country',
 'where',
 'you',
 'are',
 'located',
 'before',
 'using',
 'this',
 'ebook',
 '.',
 'title',
 'the',
 'life',
 'and',
 'adve

In [ ]:
word_counts = len(list(set(word_list)))

set_words = list(set(word_list))

word_2_int = {}

int_2_word = {}

for i,word in enumerate(set_words):

  word_2_int[word] = i

  int_2_word[i] = word

#create sequences
sequences = []

seq_length = 25

next_words = []

for i in range(0, len(word_list) - seq_length):

    words = word_list[i: i + seq_length]

    int_words = list(map(lambda word : word_2_int[word],words))

    sequences.append(int_words)

    next_words.append(word_2_int[str(word_list[i + seq_length])])

print(next_words[:5])

[3686, 807, 4224, 6047, 8]


In [ ]:
from keras.utils import to_categorical

labels = to_categorical(next_words, num_classes = word_counts)

sentences = np.array(sequences)

In [ ]:
sentences = sentences[:20000]

labels = labels[:20000]

In [ ]:

model = Sequential()

model.add(Embedding(word_counts, 100, input_length=25))

model.add(Bidirectional(LSTM(200, return_sequences = True)))

model.add(Dropout(0.3))

model.add(LSTM(200))

model.add(Dense(word_counts//2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

#model.add(BatchNormalization())

model.add(Dense(word_counts, activation='softmax'))

#model.add(BatchNormalization())

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 25, 100)           634000    
_________________________________________________________________
bidirectional_7 (Bidirection (None, 25, 400)           481600    
_________________________________________________________________
dropout_7 (Dropout)          (None, 25, 400)           0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 200)               480800    
_________________________________________________________________
dense_9 (Dense)              (None, 3170)              637170    
_________________________________________________________________
dense_10 (Dense)             (None, 6340)              20104140  
Total params: 22,337,710
Trainable params: 22,337,710
Non-trainable params: 0
__________________________________________

In [ ]:
#fit the model
#callbacks=[EarlyStopping(patience=4, monitor='val_loss'),
           #ModelCheckpoint(filepath=save_dir + "/" + 'my_model_gen_sentences_lstm.{epoch:02d}-{val_loss:.2f}.hdf5',\
                           #monitor='val_loss', verbose=0, mode='auto', period=2)]
history = model.fit(sentences, labels,
                    batch_size = 64,
                 shuffle=True,
                 epochs=100)

Epoch 1/100
20000/20000 [==============================] - 90s 4ms/step - loss: 3.7481 - accuracy: 0.2643
Epoch 2/100
20000/20000 [==============================] - 90s 5ms/step - loss: 3.6917 - accuracy: 0.2729
Epoch 3/100
20000/20000 [==============================] - 91s 5ms/step - loss: 3.6320 - accuracy: 0.2808
Epoch 4/100
20000/20000 [==============================] - 89s 4ms/step - loss: 3.5772 - accuracy: 0.2893
Epoch 5/100
20000/20000 [==============================] - 90s 4ms/step - loss: 3.5283 - accuracy: 0.2889
Epoch 6/100
20000/20000 [==============================] - 90s 5ms/step - loss: 3.4642 - accuracy: 0.3006
Epoch 7/100
20000/20000 [==============================] - 88s 4ms/step - loss: 3.4178 - accuracy: 0.3090
Epoch 8/100
20000/20000 [==============================] - 88s 4ms/step - loss: 3.3603 - accuracy: 0.3178
Epoch 9/100
20000/20000 [==============================] - 88s 4ms/step - loss: 3.3016 - accuracy: 0.3266
Epoch 10/100
20000/20000 [====================

In [ ]:

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

seed_text = "I sat holding in three branches at the top of an old oak behind my house . I could see over the top , could"

words = seed_text.lower().split()

words_number = 150
#generate the text
for i in range(words_number):
    #create the vector
    
    int_words = list(map(lambda word : word_2_int[word],words))

    pad_sequence = pad_sequences([int_words],maxlen = 25,truncating='pre')

    word_index = np.argmax(model.predict(pad_sequence)[0])

    next_word = int_2_word[word_index]

    #add the next word to the text
    seed_text += " " + next_word
    # shift the sentence by one, and and the next word at its end
    words = words[1:] + [next_word]

print(seed_text)

I sat holding in three branches at the top of an old oak behind my house . I could see over the top , could make six voyages to the east . in the ship side of slavery from the pride , ll make xury come away from the wind shifting to the southward , concluded also that if any of our men keeping close but to buy , began to recover the sails and terror and how if was many minutes weather so earnestly fortunes on and little way . we could have done within the boat and good surprised , says he , you come near the utmost barbarian , dead the lighthouse at least we were obliged to slip one of our own produce . we found the punch man to lisbon , for we had taken wrong in life which went very restrictions whatsoever . you may see visible trick happy but to make the exchange , and be convinced of their former felicity by their experience say , how just
